In [1]:
import os
from IPython.display import clear_output
from packages import esp4_import_functions as esp4fns
from numpy import zeros, shape

In [2]:
#GETTING path from user input
path = "/Users/santapile/Santas/Samples/StripeEasy/200924_BESSY_Oct20_45deg/STXM/Maxymus_Oct20/Santa/rawTest/201030/"

In [3]:
listOfOptions = ['chemical', 'magnetic']

for choice in listOfOptions:
    print(choice)

    #PROCESSING ALL BBX-FILES in the path:
    for file in os.listdir(path):
        if file.endswith(".bbx"):
            print(file)
            a = esp4fns.BBXimport(125, path + file)
            data = zeros(shape(a[0]))

            for i in range (0,len(a[0])):
                for j in range (0,len(a[0][i])):
                    for k in range (0,a[1]):
                        if choice == 'chemical':
                            y = a[0][i][j][k] #MODE1 raw data ("_%d (2).txt")
                        else:
                            y = a[0][i][j][k] / (sum(a[0][i][j]) / a[1]) #MODE2 magnetic contrast ("_%d.txt")

                        data[i, j, k] = y

            print(y)

            for k in range (0, a[1]):
                clear_output(wait=True)
                print(k+1, '/', a[1])
                if choice == 'chemical':
                    print(choice)
                    with open(path + file[:-4] + "_%d (2).csv" % k,'w+') as f: #MODE1
                        for i in range (0, len(data)):
                            for j in range (0, len(data[i])):
                                f.write(str(data[i, j, k]) + ',')
                            f.write('\n')
                else:
                    print(choice)
                    with open(path + file[:-4] + "_%d.csv" % k,'w+') as f: #MODE2
                        for i in range (0, len(data)):
                            for j in range (0, len(data[i])):
                                f.write(str(data[i, j, k]) + ',')
                            f.write('\n')
        
                    f.close()

7 / 7
magnetic
